In [8]:
import pandas as pd
import numpy as np

Answer is median of the last 8 weeks

In [13]:
train = pd.read_csv("./input/train_1.csv")
test = pd.read_csv("./input/key_1.csv")

In [14]:
test['Page'] = test.Page.apply(lambda a: a[:-11])
train['Visits'] = train[train.columns[-56:]].median(axis=1, skipna=True)

In [15]:
test = test.merge(train[['Page','Visits']], how='left')
test.loc[test.Visits.isnull(), 'Visits'] = 0

In [16]:
test[['Id','Visits']].to_csv('wiki_sub.csv', index=False)

Take weekend status into account

In [17]:
train = pd.read_csv("./input/train_1.csv")
train_flattened = pd.melt(train[list(train.columns[-49:])+['Page']], id_vars='Page', var_name='date', value_name='Visits')
train_flattened['date'] = train_flattened['date'].astype('datetime64[ns]')
train_flattened['weekend'] = ((train_flattened.date.dt.dayofweek) // 5 == 1).astype(float)

In [18]:
test = pd.read_csv("./input/key_1.csv")
test['date'] = test.Page.apply(lambda a: a[-10:])
test['Page'] = test.Page.apply(lambda a: a[:-11])
test['date'] = test['date'].astype('datetime64[ns]')
test['weekend'] = ((test.date.dt.dayofweek) // 5 == 1).astype(float)

In [19]:
train_page_per_dow = train_flattened.groupby(['Page','weekend']).median().reset_index()
test = test.merge(train_page_per_dow, how='left')
test.loc[test.Visits.isnull(), 'Visits'] = 0

In [20]:
test[['Id','Visits']].to_csv('w_sub.csv', index=False)